In [2]:
from pathlib import Path
import os
join = os.path.join
import nibabel as nib
import numpy as np
from skimage import measure
from skimage import morphology

In [30]:
# 卡阈值测试
pred_path = f"/data3/home/lishengyong/data/ssc_0802/new_slices/infer_3d/axis_1/0818_98/multi_box"
img_path = "/data3/home/lishengyong/data/ssc_0802/new_slices/3D_open_image_list"
store_path = "/data3/home/lishengyong/data/ssc_0802/new_slices/infer_3d/axis_1/0818_98/croped"
os.makedirs(store_path, exist_ok=True)
k = 1
for case in Path(pred_path).iterdir():
    mask_arr = nib.load(case).get_fdata().astype(int)
    mask_arr[mask_arr==1] = 0
    mask_arr[mask_arr==2] = 1
    img_arr = nib.load(join(img_path, case.name)).get_fdata()
    if np.sum(mask_arr * img_arr) / np.sum(mask_arr) < 200:
        crop_mask_arr = mask_arr & (img_arr < 200) & (img_arr > -750)
        nib.save(nib.Nifti1Image(crop_mask_arr.astype(float), None), join(store_path, case.name))
        k += 1
    else:
        nib.save(nib.Nifti1Image(mask_arr.astype(float), None), join(store_path, case.name))
    # if k > 50:
    #     break

In [31]:
cases = [i for i in Path(store_path).iterdir()]
crop2_path = "/data3/home/lishengyong/data/ssc_0802/new_slices/infer_3d/axis_1/0818_98/croped2"
os.makedirs(crop2_path, exist_ok=True)
for case in cases:
    mask = nib.load(case).get_fdata().astype(int)
    ori_mask = nib.load(join(pred_path, case.name)).get_fdata()
    ori_mask[ori_mask == 1] = 0
    ori_mask[ori_mask == 2] = 1
    ccs1 = measure.regionprops(measure.label(ori_mask))
    new_mask = np.zeros((32, 128, 128))
    for cc in ccs1:
        a = np.zeros((32, 128, 128), np.int8)
        a[tuple(cc.coords.T)] = 1
        b = a & mask
        single_cc = measure.label(b)
        ccs2 = measure.regionprops(single_cc)
        max_area = 0
        for cc2 in ccs2:
            if cc2.area > max_area:
                max_area = cc2.area
                coords = tuple(cc2.coords.T)
        new_mask[coords] = 1
    nib.save(nib.Nifti1Image(new_mask.astype(float), None), join(crop2_path, case.name))
    
    

In [5]:
mask_path = "/data3/home/lishengyong/data/ssc_0802/new_slices/3D_masks"
store_path = "/data3/home/lishengyong/data/ssc_0802/\
new_slices/infer_3d/axis_1/0818_98/iter5/infer_add"
os.makedirs(store_path, exist_ok=True)
for i in Path("/data3/home/lishengyong/data/ssc_0802/\
new_slices/infer_3d/axis_1/0818_98/iter5/infer_croped").iterdir():
    mask1 = nib.load(i).get_fdata()
    mask2 = nib.load(join(mask_path, i.name)).get_fdata()
    mask1[mask2 == 1] = 1
    nib.save(nib.Nifti1Image(mask1, None), join(store_path, i.name))


In [3]:
# 开运算
mask_path = "/data3/home/lishengyong/data/ssc_3d/slices/infer_3d_iter/axis_1/0818_98/iter9/infer"
store_path = "/data3/home/lishengyong/data/ssc_3d/slices/infer_3d_iter/axis_1/0818_98/iter9/opening"
os.makedirs(store_path, exist_ok=True)
#读取mask
for i in Path(mask_path).iterdir():
    image = nib.load(i).get_fdata()
    image[image == 1] = 0
    image[image == 2] = 1
    # 定义结构元素
    selem = morphology.disk(1)  # 使用半径为1的圆形结构元素
    # 进行开运算操作
    open_image_list = []
    for slice in image:
        if np.sum(slice) < 20:
            open_image_list.append(slice)
        else:
            open_image_list.append(morphology.opening(slice, selem))
    opened_image = np.stack(open_image_list)
    nib.save(nib.Nifti1Image(opened_image, None), join(store_path, i.name))

In [10]:
morphology.disk(1)

array([[0, 1, 0],
       [1, 1, 1],
       [0, 1, 0]], dtype=uint8)